# Pre-canned Queries for Python
## Function to call Government and Corporate Bond Reference Data
This query demonstrates how using a list of different identifier types and identifiers, the user can request:
  * Instrument level reference data including:
  * Instrument Identifiers
  * Coupon Schedule
  * Amount Outstanding Schedlue sorted in descending effective date order
  * Identify the Valuation Quote (the quote used to calculate market capitalisation of the issue)
  * Quote level identifiers for all active quotes of the instrument
  * The name, TRBC Scheme level 5, organizsation type, status, and ultimate parent ID for the issuer of the instrument
The query demonstrates how fields that contain enumerated values can be configured to return a human readable value rather than the enumeration Entitlements: To access the data returned by this query, the user account must have licences that include:
  * Government and Corporate Bond Reference Data - PRS / FI_GOV_REFERENCE
  * Organization Refence Data - PRS / BASIC_ENTITY
The query requests instrument identifiers associated with the instrument - if the instrument being queried  has a SEDOL or CUSIP, the Identifier will be suppressed unless the user has the appropriate licences (SYMBOLOGY / IDENTIFIER_TYPE_SEDOL and IDENTIFER_TYPE_SEDOL)


tags :     `Government`, `Corporate`, `Reference`, `Instrument`, `Organization`, `Identifiers`
## Import the library and load credentials
Credentials used by this notebook are stored in theconfig_files/credentials.ipynb. Please editcredentials.ipynb to set your credentials and run the next cell to continue with the scripts 

In [ ]:
from pathlib import Path 
import os
if Path.cwd().name != "config_files":
    p = Path("../config_files/") 
    os.chdir(p)

%run credentials.ipynb 
%run environment_selection.ipynb

To connect to the Refinitiv Data and opening a session,given function is used. To find out more about the library,you can visit [refinitiv-data library](https://pypi.org/project/refinitiv-data/)

In [ ]:
session = rdp.session.platform.Definition(
    app_key=APP_KEY,
    grant=rdp.session.platform.GrantPassword(
        username=RDP_LOGIN, password=RDP_PASSWORD
    ),
).get_session()
session.open()


## Graph call
Graph call with given sets of samples below. 

In [ ]:
%%writefile query_cache
query GovernmentCorporateBondReferenceData($identifierTypeName: GovCorpInstrumentIdentifierTypeNameEnumArgs!, $identifier: String!) {
  GovCorpInstrument(identifiers: [{identifierTypeName: $identifierTypeName, identifier: $identifier}]) {
    ObjectId
    _metadata{
      identifierTypeName
      identifier
    }
    Reference {
      SecurityDescription
      MaturityDate
      CouponTypeCode
      CouponTypeCodeDescription: CouponTypeCode(format: Name, language: en_US, style: LongName)
      Cic
      CfiCode
    }
    AssetStatus {
      AssetStatusCode
      AssetStatusCodeDescription: AssetStatusCode(format: Name, language: en_US, style: LongName)
      EffectiveFromDate
    }
    CouponRateSchedule {
      CouponRate
      EffectiveFromDate
      InterestLegNumber
      LastChangedDate
    }
    IssuanceDefinition {
      DatedDate
      IssueCurrencyIsoCode
      IssueAmount
    }
    InterestDefinition {
      DayCountCode_v2
      DayCountCodeDescription: DayCountCode_v2(format: Name, language: en_US, style: LongName)
    }
    AmountOutstandingHistory(sort: {EffectiveFromDate: DESC}) {
      CurrencyIsoCode
      TotalOutstandingAmount
      EffectiveFromDate
      NetChangeCode
      NetChangeCodeDescription: NetChangeCode(format: Name, language: en_US, style: LongName)
    }
    Identifiers {
      EffectiveFromDate
      IdentifierCode
      IdentifierCodeDescription:IdentifierCode(format:Name,language:en_US,style:LongName)
      IdentifierValue
    }
    Quotes {
      ObjectId
      Identifier(filter: {AND: [{IdentifierTypeCode: {IN: ["ORC", "RIC"]}}, {QuoteIsActiveFlag: {EQ: "y"}}]}) {
        GlobalRegionCode
        IdentifierTypeCode
        IdentifierValue
        ObjectTypeCode
        QuoteContributorCode
        QuoteCurrencyIsoCode
        QuoteIsActiveFlag
        QuotePriceSourceCode
        QuotePriceSourceCodeDescription: QuotePriceSourceCode(format: Name, language: en_US, style: LongName)
      }
    }
    Issuer {
      ObjectId
      Names {
        Name {
          OrganizationName(filter: {NameTypeCode: {EQ: "LNG"}}) {
            Name
            NameTypeCode
            NameTypeCodeDescription: NameTypeCode(format: Name, language: en_US, style: LongName)
        }
        }
      }
      ClassificationTrbcPrimaryScheme_v2 {
        Industry {
          Name
        }
        Identifier {
          Identifiers(filter: {ValueTypeId: {EQ: "320159"}}) {
            Value
            ValueTypeCode
            ValueTypeId
          }
        }
      }
      Details {
        OrganizationDetails {
          JurisdictionOfIncorporation
          OrganizationTier
          OrganizationTypeCode(format: Name, language: en_US, style: LongName)
          OrganizationSubTypeCode(format: Name, language: en_US, style: LongName)
          OrganizationStatusCode
          OrganizationStatusCodeDescription: OrganizationStatusCode(format: Name, language: en_US, style: LongName)
        }
      }
      UltimateParentId
    }
  }
}


## Sample :  Query demonstrates how using a list of ISINs

In [ ]:

variables = {'identifierTypeName': 'Isin', 'identifier': 'US645767AW49'}

In [ ]:
%run config_request_handler.ipynb

In [3]:
%run code_execution.ipynb

 
    data: { 
        GovCorpInstrument: [ 
            { 
                ObjectId: 15628303271, 
                _metadata: { 
                    identifierTypeName: Isin, 
                    identifier: US645767AW49 
                }, 
                Reference: { 
                    SecurityDescription: VZ     7.850 11/15/29 PUT, 
                    MaturityDate: 2029-11-15, 
                    CouponTypeCode: FXPV, 
                    CouponTypeCodeDescription.................... 
......................................... 


## Sample :  Query demonstrates how using a list of Sedols

In [ ]:

variables = {'identifierTypeName': 'Sedol', 'identifier': 'B4333T3'}

In [ ]:
%run config_request_handler.ipynb

In [3]:
%run code_execution.ipynb

reserved_to_be_populated

## Sample :  Query demonstrates how using a list of Cusips

In [ ]:

variables = {'identifierTypeName': 'Cusip', 'identifier': '36966TDN9'}

In [ ]:
%run config_request_handler.ipynb

In [3]:
%run code_execution.ipynb

reserved_to_be_populated

## Close the default session when done

In [ ]:
session.close()